In [1]:
# movie linear regression
import pandas as pd
import numpy as np
import text_process as tp

In [2]:
dataset = pd.read_csv(r'../../data/movie/movie_reviews.csv')

In [3]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

In [6]:
train_reviews = reviews[:35000]
train_sentiments = sentiments[:35000]
test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]
norm_train_reviews = tp.normalize_corpus(train_reviews)
norm_test_reviews = tp.normalize_corpus(test_reviews)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


In [9]:
tfidf = TfidfVectorizer(max_features=2000)
train_features = tfidf.fit_transform(norm_train_reviews)

In [11]:
test_features = tfidf.transform(norm_test_reviews)

In [12]:
lr = LogisticRegression()
lr.fit(train_features, train_sentiments)
predictions = lr.predict(test_features) 

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [13]:
from sklearn import metrics
metrics.accuracy_score(test_sentiments, predictions)

0.8812666666666666

In [38]:
# online learning

import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
import csv
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            punkt = line.rfind(",")
            text, label = line[:punkt], line[punkt+1:-1]
            yield text, label

In [45]:
next(stream_docs(path=r'../../data/movie/movie_reviews.csv'))

('"One of the other reviewers has mentioned that after watching just 1 Oz episode you\'ll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the

In [40]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [41]:
doc_stream = stream_docs(path=r'../../data/movie/movie_reviews.csv')

In [46]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier


vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)

In [48]:
classes = np.array(["negative", "positive"])
for _ in range(4):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train,classes=classes)

In [51]:
import pickle
pickle.dump(stop, open('stopwords.pkl', 'wb'), protocol=4)   
pickle.dump(clf, open('classifier.pkl', 'wb'), protocol=4)